## Documentation

To read more about the update API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update.html).

![update_api_docs](../images/update_api_docs.png)

## Connect to ElasticSearch


In [2]:
import { Client } from "npm:@elastic/elasticsearch";

const client = new Client({
  node: "http://localhost:9200",
  auth: {
    apiKey: "cnhtNzRaVUJoZTlWUmNVWWJFbGg6SktrbkY4aFlUSmE5d2dwbGJCUXdEZw",
  },
});

const info = await client.info();
console.log(info);


{
  name: "ff84884348ad",
  cluster_name: "docker-cluster",
  cluster_uuid: "pv2aBcEGSgGBq5GJpCBNHQ",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [3]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `documentIds` list.


In [4]:
import data from "../data/dummy_data.json" with { type: "json" };

const documentIds = [];
data.forEach(async (document: any) => {
  const response = await client.index({
    index: "my_index",
    document,
  });

  documentIds.push(response._id);
});


In [5]:
documentIds

[
  "k6-tAJYBIA2L3Kcm21Z5",
  "lK-tAJYBIA2L3Kcm21aP",
  "kq-tAJYBIA2L3Kcm21Z5"
]

## Update API

### 1. If documents exists in the index

If both `doc` and `script` are specified, then `doc` is ignored. If you specify a scripted update, include the fields you want to update in the script.

#### 1.1 Update an existing field

##### 1.1.1 Method 1 (Doc)

In [6]:
await client.update({
  index: "my_index",
  id: documentIds[0],
  doc: {
    title: "Updated Title",
  },
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 2,
  result: "updated",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 3,
  _primary_term: 1
}

In [7]:
await client.get({
  index: "my_index",
  id: documentIds[0],
})

{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 2,
  _seq_no: 3,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Updated Title",
    text: "Here is another example of a document.",
    createdAt: "2025-03-02"
  }
}

##### 1.1.2 Method 2 (Script)

In [8]:
await client.update({
  index: "my_index",
  id: documentIds[1],
  script: {
    source: "ctx._source.title = params.title",
    params: {
      title: "Updated Title 2",
    },
  }
})

{
  _index: "my_index",
  _id: "lK-tAJYBIA2L3Kcm21aP",
  _version: 2,
  result: "updated",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 4,
  _primary_term: 1
}

In [9]:
await client.get({
  index: "my_index",
  id: documentIds[1],
});


{
  _index: "my_index",
  _id: "lK-tAJYBIA2L3Kcm21aP",
  _version: 2,
  _seq_no: 4,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Updated Title 2",
    text: "The content of the third document goes here.",
    createdAt: "2025-03-03"
  }
}

#### 1.2 Add a new field

To add a new field, you can either use the `script` argument or the `doc` argument.

##### 1.2.1 Method 1 (Doc)

In [10]:
await client.update({
  index: "my_index",
  id: documentIds[0],
  doc: {
    newFieldDoc: "New Field Doc",
  },
});

{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 3,
  result: "updated",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 5,
  _primary_term: 1
}

In [11]:
await client.get({
  index: "my_index",
  id: documentIds[0],
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 3,
  _seq_no: 5,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Updated Title",
    text: "Here is another example of a document.",
    createdAt: "2025-03-02",
    newFieldDoc: "New Field Doc"
  }
}

##### 1.2.2 Method 2 (Script)

In [12]:
await client.update({
  index: "my_index",
  id: documentIds[0],
  script: {
    source: "ctx._source.newFieldScript = params.newFieldScript",
    params: {
      newFieldScript: "New Field Script",
    },
  },
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 4,
  result: "updated",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 6,
  _primary_term: 1
}

In [13]:
await client.get({
  index: "my_index",
  id: documentIds[0],
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 4,
  _seq_no: 6,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Updated Title",
    text: "Here is another example of a document.",
    createdAt: "2025-03-02",
    newFieldDoc: "New Field Doc",
    newFieldScript: "New Field Script"
  }
}

#### 1.3 Remove a field

In [14]:
await client.update({
  index: "my_index",
  id: documentIds[0],
  script: {
    source: "ctx._source.remove('newFieldDoc')",
  },
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 5,
  result: "updated",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 7,
  _primary_term: 1
}

In [15]:
await client.get({
  index: "my_index",
  id: documentIds[0],
});


{
  _index: "my_index",
  _id: "k6-tAJYBIA2L3Kcm21Z5",
  _version: 5,
  _seq_no: 7,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Updated Title",
    text: "Here is another example of a document.",
    createdAt: "2025-03-02",
    newFieldScript: "New Field Script"
  }
}

### 2. If documents doesn't exist in the index

We use `doc_as_upsert` to tell Elasticsearch that if the document does not exist, it should be inserted as a new document.

In [16]:
await client.update({
  index: "my_index",
  id: 'does_not_exist',
  doc: {
    title: "New Document",
    text: "This is a new document",
  },
  doc_as_upsert: true,
});


{
  _index: "my_index",
  _id: "does_not_exist",
  _version: 1,
  result: "created",
  _shards: { total: 2, successful: 1, failed: 0 },
  _seq_no: 8,
  _primary_term: 1
}

Count documents after upsert

In [17]:
await client.count({
  index: "my_index",
});


{
  count: 4,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 }
}